In [1]:
from pinecone import Pinecone, ServerlessSpec
PINECONE_API_KEY="pcsk_5SakzS_Rr4yjB6AVBwSEj7vGVfeFsgzEgQ5t6TouFjH6MPe154GNLAVSMwPqpL6RdZgXAU" # API key
pc = Pinecone(api_key=PINECONE_API_KEY) # connect to pinecone

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
index_name = "language" # index name
dimension = 2  # Количество измерений в векторе (2 параметра)
metric = "cosine"  # Сходство на основе косинуса
# creat index at the first time
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric=metric,
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
  )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'cc572dc5df7cf300a62eff455b17ab95', 'Date': 'Tue, 05 Nov 2024 09:42:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import pandas as pd
data = pd.read_csv('merged_data_csv_1_exported_1.csv')
data = data.dropna(subset=['Total_words', 'Unique_words']) # drop na values 
data = data.fillna("") # fill na as string
vectors = []
    
for _, row in data.iterrows():
    # Вектор для Pinecone — только `Total_words` и `Unique_words`
    values = [float(row['Total_words']), float(row['Unique_words'])]  
    metadata = {
        "pastEnglishLearningMethodChild": row['pastEnglishLearningMethodChild'],
        "PresentEnglishLearningMethodChild": row['PresentEnglishLearningMethodChild'],
        "grade": row['grade'],
        "year_birth": row['year_birth'],
        "game_experience": row['game_experience'],
        "gender": row['gender'],
        "device": row['device'],
        "group": row['group'],
        "lang_level": row['lang_level'],
        "age": row['age']
    }
    # append to the vector
    vectors.append({
        "id": str(row['ID_']),
        "values": values,
        "metadata": metadata
    })

In [ ]:
#load data to db
index = pc.Index(index_name)
index.upsert(
    vectors=vectors,
    namespace="test_group"
)

{'upserted_count': 89}

In [29]:
print(index.describe_index_stats())

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'test_group': {'vector_count': 84}},
 'total_vector_count': 84}


In [2]:
#search similar data
index_name = "language" # index name
index = pc.Index(index_name)
# dimension = 2  # 
# total = float(330)
# uniq = float(126)
# vector = [total, uniq]
# results = index.query(
#     namespace="test_group",
#     vector=vector,
#     top_k=3,
#     include_values=False,
#     include_metadata=True,
#     filter={
#         "age": {"$gte": 8, "$lte": 9} 
#     }
# )

# print(results)

def get_similar_user(total_words, unique_words, age, gender):
    vector = [float(total_words), float(unique_words)]
    results = index.query(
        namespace="test_group",
        vector=vector,
        top_k=3,
        include_values=False,
        include_metadata=True,
        filter={
            "age": {"$gte": age-1, "$lte": age+1} ,
            "gender": gender
        }
    )
    return results

# get_similar_user(330, 126, 8)

In [3]:
get_similar_user(223, 115, 8, "남")

{'matches': [{'id': '2292',
              'metadata': {'PresentEnglishLearningMethodChild': '영어학습 프로그램을 '
                                                                '활용(웅진 스마트올)',
                           'age': 8.0,
                           'device': 'Mobile,"콘솔 (플레이스테이션, X-Box등)"',
                           'game_experience': '네',
                           'gender': '남',
                           'grade': '초등 2학년',
                           'group': 'Kids',
                           'lang_level': '',
                           'pastEnglishLearningMethodChild': '학습경험없음',
                           'year_birth': 2016.0},
              'score': 0.999076962,
              'values': []},
             {'id': '2294',
              'metadata': {'PresentEnglishLearningMethodChild': '원서읽기',
                           'age': 9.0,
                           'device': 'Mobile',
                           'game_experience': '네',
                           'gender': '남',
             